EPITA 2023 IML lab02_clustering_03-segmentation v2023-03-27_103406 by G. Tochon & J. Chazalon

<div style="overflow: auto; padding: 10px; margin: 10px 0px">
<img alt="Creative Commons License" src='img/CC-BY-4.0.png' style='float: left; margin-right: 20px'>
    
This work is licensed under a [Creative Commons Attribution 4.0 International License](http://creativecommons.org/licenses/by/4.0/).
</div>

# Lab 2, part 3: Image segmentation using clustering (naive)

In this part we will apply clustering the segment image into homogeneous regions, as illustrated by the figure below.

![](img/segmentation.png)

This is a naive segmentation version, which is not as elaborated as the [SLIC](https://www.epfl.ch/labs/ivrl/research/slic-superpixels/) approach.
However, it illustrates well how to combine color and position information within the same clustering process.

We put a couple of images from the [Berkeley segmentation dataset](https://www2.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/segbench/) (BSDS500) under the `data/` directory.

The trick is pretty simple here:
1. we consider each pixel from the image as points in a 3D color space (RGB);
2. we can add some scaled ($\in [0,1]$) coordinates for each pixel in the image to add spatial information and encourage the clustering to be more consistent with neighbor pixels.

We will guide you throughout this first application of clustering for image segmentation.

The goal here are to:
- try various off-the-shelf clustering algorithms from scikit-learn;
- get some intuition about their strengths, weaknesses and use-cases;
- learn to generate mesh grids with numpy;
- make some nice illustrations;
- start to think about computer vision problems like: 
  - How would you filter the contours generated in the previous figure?
  - What would be the problem with using the HSV color space?

## 0. Setup

### Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn as sk
import skimage as ski
import skimage.io as skio

### Data loading

In [ ]:
# Feel free to use another image!
img = skio.imread("data/300091.jpg")
img.shape, img.dtype

In [ ]:
plt.imshow(img)

## 1. Simple approach
For this first step, we will use color information only. We do not integrate any spatial information in the features (so pixels features = colors).

Using a better color space than RGB should be interesting to try too, but let's keep things simple here.
Always start with the dumbest possible pipeline.

### 1.1. Predict pixel cluster ids

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

<b>Using `sklearn.cluster.MiniBatchKMeans`, learn a clustering predictor and predict the cluster ids for each pixel of the image.</b>

<b>Hints</b>
- Use a low number of clusters (like 6).
- Set the `random_state` parameter for reproducibility.

</div>

In [ ]:
# let's be more serious, use the mini batch version
# FIXME
# from sklearn.cluster import ???

# ...

# labels = clusterer.predict(X)

###  1.2. Visualization

We can now rearrange the labels into a 2D structure of the same width and height as the original image, and display it.

In [ ]:
plt.imshow(labels.reshape(img.shape[:2]))

Because colormaps are designed to give similar colors to similar values, the result is not very pleasant to view.

What we want is a way to assign very different colors to each cluster of pixels in our image.

To do so, we will **build a [lookup table](https://en.wikipedia.org/wiki/Lookup_table) (LUT) to manually assign an adequate color (RGB triplet) to each label value (sequential integers).**

In [ ]:
import matplotlib.cm as cm
# To learn more about color maps, bookmark this page
# https://matplotlib.org/stable/tutorials/colors/colormaps.html

In [ ]:
def random_lut(n_values):
    '''Build a random LUT for `n_values` elements (sequential integers).'''
    samples = np.linspace(0, 1, n_values)  # take n_values values between 0 and 1 (evenly spaced)
    rng = np.random.default_rng(3)  # get a RNG with a specific seed
    samples = rng.permutation(samples)  # shuffle our values
    colors = cm.hsv(samples, alpha=None, bytes=True)  # get corresponding colors from the HSV color map
    return colors[...,:3]  # remove alpha channel and return

In [ ]:
# Generate a random LUT for `n_clusters` elements
lut = random_lut(n_clusters)
lut

This reads as follows:  
(*run next cells!*)

In [ ]:
from IPython.display import HTML

In [ ]:
_html_data = ["<ul>"]
for ii, row in enumerate(lut):
    _html_data.append(
        f"<li>row <code>{ii}</code>, "
        f"which will be used of pixels with label <code>{ii}</code>, " 
        f"will get the triplet <code>{tuple(row)}</code>, "
        f"which correspond to <span style='color: rgb{tuple(row)}'>this color</span></li>")
_html_data.append("</ul>")
HTML("".join(_html_data))

And now, numpy indexing black magick happens!

For each element (integer cluster id) in `labels`, we pick the corresponding color in `lut`!

In [ ]:
recolored = lut[labels]
recolored.shape

We could either reshape `labels` to the shape (row and columns) of the original image, or we can reshape the `recolored` content later, as below.

Please feel free to improve this code to your taste!

In [ ]:
def show_image_seg(img, recolored, n_clusters):
    plt.figure(figsize=(12,4))
    plt.subplot(1,2,1)
    plt.imshow(img)
    plt.title("Original image")
    plt.subplot(1,2,2)
    plt.imshow(recolored.reshape(img.shape))
    plt.title(f"Segmented areas (with spatial info), {n_clusters} clusters")

In [ ]:
show_image_seg(img, recolored, n_clusters)

This is very pretty, but we somehow lack spatial consistency here…

## 2. Integrate spatial information

### 2.1. Data preparation
Here you will need to add some spatial information to your pixels to obtain a better spatial consistency in the predictions.

Do do so, we will add horizontal and vertical coordinates to each pixel in the image. We will scale these values to have homogeneous features domains (everything in $[0,1]$).

First we will create 2 new channels (one with x values and one with y values) for each pixel, then we will create an image with 5 scalars for each pixel: $(R, G, B, X, Y)$, all these values being scaled.

Top-left pixel will have the following value: $(R_{00}, G_{00}, B_{00}, 0.0 , 0.0)$ and the bottom right pixel will have the following values: $(R_{HW}, G_{HW}, B_{HW}, 1.0 , 1.0)$ where $H$ (resp. $W$) is the height (resp. width) of the image.

**The final image must have the following shape: `(original_img.shape[0], original_img.shape[1], 5)`.**

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

<b>Using `np.meshgrid`, generate two new channels for the image: one with the x coordinate for each pixel, and one for the y coordinate for each pixel.</b>

</div>

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

<b>Now scale the color values of your image between 0 and 1 to get homogeneous features.</b>

</div>

Let us scale our image color values between 0 and 1 to facilitate our work.

In [ ]:
img_scaled = img / 255

Please note that we can display either uint8 or float pixel values with Matplotlib!

In [ ]:
print(f"data type: {img_scaled.dtype}, min value: {img_scaled.min()}, max value: {img_scaled.max()}")
plt.imshow(img_scaled);

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

<b>Finally, use `np.concatenate` to create the final image. Make sure you check the shape, data type and value domains for the result!</b>

</div>

In [ ]:
# FIXME
# img_sp = ...  # image with SPatial information
# img_sp.shape, img_sp.dtype, ...

### 2.2. Try `MiniBatchKMeans` again

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

<b>Run the same experiment as in section 1: are you happier with the results?</b>

</div>

Much more object consistency, right?

### 2.3. Try a Gaussian Mixture

Let us try again with a more elaborate model.

This time we will use a Gaussian Mixture (that we will study further later).

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

<b>Run the same experiment using `sklearn.mixture.GaussianMixture`, and comment the results. Try various number of clusters.</b>

<i>Hint: don't forget to regenerate a LUT if you have a new number of clusters.</i>
    
</div>

In [ ]:
from sklearn.mixture import GaussianMixture

In [ ]:
# TODO

### 2.4. Try Mean-shift
Mean-shift is a kind of non-parametric approach.
There is a pretty nice tutorial about it for OpenCV:
https://docs.opencv.org/master/d7/d00/tutorial_meanshift.html

Scikit-learn also has an implementation at `sklearn.cluster.MeanShift`.

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

<b>Run the same experiment using `sklearn.cluster.MeanShift`, and comment the results. Use a small bandwith.</b>

<i>Hint: Here you do not control the number of clusters but the radius of the ball around which data is considered. As every feature $\in [0,1]$ we should use a rather small value like $0.2$.</i>
    
</div>

### 2.5. BIRCH
Hey, there are many more clustering technique down here!

The illustration of the introduction was generated using BIRCH, but another technique could have been used…

<div style="overflow: auto; border-style: dotted; border-width: 1px; padding: 10px; margin: 10px 0px">
<img alt="work" src='img/work.png' style='float: left; margin-right: 20px'>

<b>Why not trying more classifiers if you have time?</b>
    
</div>

### 2.7. Nice figure

As a gift, you can re-generate the beautiful figure from the introduction 🙃!

In [ ]:
# Create a nice figure to illustrate the introduction of the notebook
plt.figure(figsize=(16,11))
plt.subplot(2,2,1)
plt.imshow(img)
plt.title("Original image")
plt.axis("off")
plt.subplot(2,2,2)
plt.imshow(recolored.reshape(img.shape))
plt.title(f"Segmented areas (with spatial info.), {n_clusters} clusters")
plt.axis("off")
plt.subplot(2,2,3)
plt.imshow(ski.color.rgb2gray(img), cmap='gray')
plt.contour(labels.reshape(img.shape[:2]), alpha=0.5, linewidths=1, colors='r')
plt.title("Contours (red) over original image (gray-level)")
plt.axis("off")
plt.subplot(2,2,4)
plt.contour(labels.reshape(img.shape[:2]), 
            linewidths=0.5, colors='k',
            origin='image')
plt.title("Contours only")
plt.gca().set_aspect("equal")  # works better than # plt.axis("equal")
plt.axis("off")
plt.tight_layout();

### 2.8. Hierarchical Agglomerative Clustering

We have too much data to try a HAC directly…

In [ ]:
# from sklearn.cluster import AgglomerativeClustering

In [ ]:
# n_clusters = 8
# for linkage in ('single', 'average', 'complete', 'ward',):
#     clusterer = AgglomerativeClustering(linkage=linkage, n_clusters=n_clusters)
#     n_features = 5
#     labels = clusterer.fit_predict(img_sp.reshape((-1,n_features)))
#     recolored = random_lut(n_clusters)[labels]
#     plt.figure(figsize=(12,4))
#     plt.subplot(1,2,1)
#     plt.imshow(img)
#     plt.title("Original image")
#     plt.subplot(1,2,2)
#     plt.imshow(recolored.reshape(img.shape))
#     plt.title(f"Segmented areas (with spatial info), {n_clusters} clusters, {linkage} linkage")

MemoryError: Unable to allocate 88.8 GiB for an array with shape (11919757200,) and data type float64


# Good job!
You are now ready to move on to the next part.